# Image_Classification_Pytorch (Inference)

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [定義模型方法](#5)
1. [製作資料集＆資料擴增＆推論模型](#6)
1. [待辦事項](#7)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [1]:
!pip3 install git+https://github.com/rwightman/pytorch-image-models.git

  Cloning https://github.com/rwightman/pytorch-image-models.git to /tmp/pip-req-build-dujb_g03
  Running command git clone -q https://github.com/rwightman/pytorch-image-models.git /tmp/pip-req-build-dujb_g03


In [2]:
# 資料處理套件
import os
import gc
import cv2
import sys
import time
import timm
import random
import numpy as np
import pandas as pd
import albumentations as A

from tqdm import tqdm
from albumentations.pytorch.transforms import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

In [3]:
# pytorch深度學習模組套件
import torch
import torch.nn as nn

from torch.utils.data import Dataset, DataLoader
from torchvision import models

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [4]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

device(type='cpu')

In [5]:
# 查看pytorch版本
print(torch.__version__)

1.7.0


In [6]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'resized-2015-2019-diabetic-retinopathy-detection/' 

# (String)CSV根路徑
CSV_ROOT_PATH = PATH+r'aptos2019-blindness-detection/' 

# (String)測試資料路徑
TEST_DATA_PATH = DATA_ROOT_PATH+r'resized_test19/'

# (String)測試CSV路徑
TEST_CSV_PATH = CSV_ROOT_PATH+r'sample_submission.csv'

# (Boolean)是否要匯入Library
IMPORT_PYTORCH_LIBRARY = False

# (String)Library的路徑
PYTORCH_LIBRARY_PATH = PATH + "PyTorch_Library/"

# (String)讀取預訓練模型/權重的名稱，當fold model時，後面會自動加_NUMBER
LOAD_MODEL_NAME = ['efficientnet_v2_b0', 'efficientnet_v2_b0', 'efficientnet_v2_b0']

# (String)讀取預訓練模型/權重的儲存路徑
LOAD_MODEL_PATH = [PATH + r'test1111/', 
                   PATH + r'test1111/', 
                   PATH + r'test1111/']

In [7]:
if DEVICE != torch.device("cpu"):
    !nvidia-smi

In [8]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    
if IMPORT_PYTORCH_LIBRARY:
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Loss.py")
    sys.path.append(PYTORCH_LIBRARY_PATH + "Custom_Model.py")

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [9]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 5

# (Int)集成模型數量
ENSEMBLE_MODEL_COUNT = 3

# (Int List)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = [1]*ENSEMBLE_MODEL_COUNT

# (Int List)圖片尺寸
IMAGE_SIZE = [224]*ENSEMBLE_MODEL_COUNT

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位
IMAGE_NAME = 'id_code'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = False

#  (Boolean)圖像轉為RGB
COLOR_CONVERT_RGB = True

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

# (Boolean)是否資料轉Tensor時啟動鎖頁內存(GPU內存)，而不鎖頁內存就是會使用到硬碟虛擬內存
PIN_MEMORY = False

# (Int)要用於數據加載的子進程數。0表示將在主進程中加載數據。（默認值：0）
NUM_WORKERS = 0
    
# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定

資料擴增教學
https://zhuanlan.zhihu.com/p/107399127

資料擴增Doc
https://vfdev-5-albumentations.readthedocs.io/en/docs_pytorch_fix/api/augmentations.html
'''

# (Float)訓練集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_TRAIN_TRANSFORMS = 1.0

# (Float)驗證集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VAL_TRANSFORMS = 1.0

# 以下資料擴增為訓練集使用=============================================

# (Float)模糊的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_BLUR = 0

# (Int)模糊的上限
BLUR_LIMIT = 3

# (Float)水平翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HORIZONTALFLIP = 0

# (Float)垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VERTICALFLIP = 0

# (Float)水平和垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_FLIP = 0

# (Float)隨機旋轉90度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMROTATE90 = 0

# (Float)平移縮放旋轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_SHIFTSCALEROTATE = 0

# (Float)平移縮放旋轉的平移上限
SHIFTSCALEROTATE_SHIFT_LIMIT = 0.0625

# (Float)平移縮放旋轉的縮放上限
SHIFTSCALEROTATE_SCALE_LIMIT = 0.1

# (Float)平移縮放旋轉的旋轉上限
SHIFTSCALEROTATE_ROTATE_LIMIT = 45

# (Float)彈性變換的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_ELATICTRANSFORM = 0

# (Float)彈性變換的alpha高斯過濾參數
ELATICTRANSFORM_ALPHA = 1

# (Float)彈性變換的sigma高斯過濾參數
ELATICTRANSFORM_SIGMA = 50

# (Float)彈性變換的alpha_affine，範圍為（-alpha_affine，alpha_affine）
ELATICTRANSFORM_ALPHA_AFFINE = 50

# (Float)網格失真的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_GRIDDISTORTION = 0

# (Int)網格失真的每一條邊上網格單元數量
GRIDDISTORTION_NUM_STEPS = 5

# (Float)隨機亮度對比度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMBRIGHTNESSCONTRAST_CONTRAST = 0

# (Float)隨機亮度的上限
RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT = 0.2

# (Float)隨機對比度的上限
RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT = 0.2

# (Float)隨機色調飽和度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HUESATURATIONVALUE = 0

# (Float)隨機色調飽和度的色調上限
HUESATURATIONVALUE_HUE_SHIFT_LIMIT = 20

# (Float)隨機色調飽和度的飽和度上限
HUESATURATIONVALUE_SAT_SHIFT_LIMIT = 30

# (Float)隨機色調飽和度的值上限
HUESATURATIONVALUE_VAL_SHIFT_LIMIT = 20

# (Float)對比度受限自適應直方圖均衡的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_CLAHE = 0

# (Float)對比度受限自適應直方圖均衡的對比度上限
CLAHE_CLIP_LIMIT = 4.0

# (Float)隨機在圖像上生成黑色矩形的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_COARSEDROPOUT = 0

# (Int)隨機在圖像上生成黑色矩形的數量
COARSEDROPOUT_NUM_HOLES = 8

# (Int)隨機在圖像上生成黑色矩形的最大高度
COARSEDROPOUT_MAX_H_SIZE = 8

# (Int)隨機在圖像上生成黑色矩形的最大寬度
COARSEDROPOUT_MAX_W_SIZE = 8

# (Float)隨機縮放剪裁的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMRESIZEDCROP = 0

# (Float Tuple)隨機縮放剪裁之前的圖像比例縮放
RANDOMRESIZEDCROP_SCALE = (0.08, 1.0)

# (Int)隨機縮放剪裁之前的圖像高度
RANDOMRESIZEDCROP_HEIGHT = IMAGE_SIZE

# (Int)隨機縮放剪裁之前的圖像寬度
RANDOMRESIZEDCROP_WIDTH = IMAGE_SIZE

# 以上資料擴增為訓練集使用=============================================

# 以下資料擴增為訓練集和驗證集共用======================================

# (Float)縮放的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RESIZE = 1.0

# (Int)縮放後的圖片高度
RESIZE_HEIGHT = IMAGE_SIZE

# (Int)縮放後的圖片寬度
RESIZE_WIDTH = IMAGE_SIZE

# (Float)正規化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_NORMALIZE = 1.0

# (Float List)正規化的平均值([0,1]的參考平均值:[0.485, 0.456, 0.406], [-1,1]的參考平均值:[0.5, 0.5, 0.5])
NORMALIZE_MEAN = [0.485, 0.456, 0.406]

# (Float List)正規化的標準差([0,1]的參考標準差[0.229, 0.224, 0.225], [-1,1]的參考標準差[0.5, 0.5, 0.5])
NORMALIZE_STD = [0.229, 0.224, 0.225]

# (Float)正規化的PIXEL最大值(參考PIXEL最大值255.0)
NORMALIZE_MAX_PIXEL_VALUE = 255.0

# (Float)歸一化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
# ToTensorV2()將[0, 255]的PIL.Image或[H, W, C]的numpy.ndarray數據，
# 轉換為形狀[C, H, W]的torch.FloadTensor，並歸一化。
P_TOTENSORV2 = 1.0

# 以上資料擴增為訓練集和驗證集共用======================================


''''模型參數設定'''

# (String List)模型載入方式(MODEL、WEIGHT_OF_CUSTOM_MODEL、WEIGHT_OF_TIMM_MODEL、WEIGHT_OF_BASE_MODEL）
MODEL_LIST = ["WEIGHT_OF_TIMM_MODEL"]*ENSEMBLE_MODEL_COUNT

if "WEIGHT_OF_CUSTOM_MODEL" in MODEL_LIST:
    # (Model List)模型載入方式有CUSTOM_MODEL，依照出現次數順序列出
    CUSTOM_MODEL = [None]*ENSEMBLE_MODEL_COUNT

if "WEIGHT_OF_TIMM_MODEL" in MODEL_LIST:
    # (Model List)模型載入方式有TIMM_MODEL，依照出現次數順序列出
    TIMM_MODEL = ["tf_efficientnet_b0"]*ENSEMBLE_MODEL_COUNT

if "WEIGHT_OF_BASE_MODEL" in MODEL_LIST:
    # (Model List)模型載入方式有BASE_MODEL，依照出現次數順序列出
    BASE_MODEL = [models.resnet18]*ENSEMBLE_MODEL_COUNT

# (Boolean)模型是否使用分類層輸出，否則為全連接層
CLASSIFIER_OUTPUT = [True]*ENSEMBLE_MODEL_COUNT

# (Float)Dropout比率
DROPOUT = [1]*ENSEMBLE_MODEL_COUNT

# (Boolean)Bias偏移量
BIAS = [True]*ENSEMBLE_MODEL_COUNT

# (Boolean)是否印出完整模型
MODEL_PRINT = [False]*ENSEMBLE_MODEL_COUNT


''''訓練參數設定'''

# (Int List)每批推論的尺寸
BATCH_SIZE = [256]*ENSEMBLE_MODEL_COUNT

In [10]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [11]:
print('Reading data...')

# 讀取訓練資料集CSV檔
test_csv = pd.read_csv(TEST_CSV_PATH,encoding="utf8")

print('Reading data completed')

Reading data...
Reading data completed


In [12]:
# 顯示訓練資料集CSV檔
test_csv.head()

,id_code,diagnosis
0,0005cfc8afb6,0
1,003f0afdcd15,0
2,006efc72b638,0
3,00836aaacf06,0
4,009245722fa4,0


In [13]:
print("Shape of train_data :", test_csv.shape)

Shape of train_data : (1928, 2)


# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [14]:
class build_model(nn.Module):

    def __init__(self, count, custom_model_count, timm_model_count, base_model_count):
        super().__init__()
        
        if MODEL_LIST[count] == "MODEL":
            # 載入預訓練模型
            self.model = torch.load(LOAD_MODEL_PATH[count])
        elif MODEL_LIST[count] == "WEIGHT_OF_CUSTOM_MODEL":
            # 載入模型架構
            self.model = CUSTOM_MODEL[custom_model_count]
        elif MODEL_LIST[count] == "WEIGHT_OF_TIMM_MODEL":
            self.model = timm.create_model(BASE_MODEL[timm_model_count], pretrained = None)
        else:
            self.model = BASE_MODEL[base_model_count](pretrained = None)

        if MODEL_LIST[count] != "MODEL":
            # 載入預訓練權重
            model.load_state_dict(LOAD_MODEL_PATH[count])

        if CLASSIFIER_OUTPUT[count]:
            n_features = self.model.classifier.in_features
            self.model.classifier = nn.Sequential(nn.Dropout(DROPOUT[count]), nn.Linear(n_features, CLASSES, bias = BIAS[count]))
        else:
            n_features = self.model.fc.in_features
            self.model.fc = nn.Sequential(nn.Dropout(DROPOUT[count]), nn.Linear(n_features, CLASSES, bias = BIAS[count]))

    def forward(self, x):
        x = self.model(x)
        return x

# 6. 製作資料集＆資料擴增＆推論模型<a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

In [15]:
class MyDataset(Dataset):
    def __init__(self, df, transforms = None):
        super().__init__()
        self.df = df.reset_index(drop=True).copy()
        self.transforms = transforms
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index: int):
        image_name = self.df[IMAGE_NAME].values[index]

        if IMAGE_NAME_HAVE_EXTENSION:
            image_path = TEST_DATA_PATH + image_name
        else:
            image_path = TEST_DATA_PATH + image_name + IMAGE_NAME_EXTENSION
            
        image = cv2.imread(image_path)
        
        if COLOR_CONVERT_RGB:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transforms is not None:
            image = self.transforms(image = image)['image']
            
        return image

In [16]:
# 確定是否將應用此增強。機率為 p = 1.0 意味著我們總是從上面應用轉換。
# p = 0 將意味著將忽略轉換塊。
# 0 < p < 1.0 等於每個擴增都具有以一定概率應用的選項。
# OneOf 隨機選取一種增強擴增

def get_test_transforms(fold):
    return A.Compose([
        A.Blur(blur_limit = BLUR_LIMIT, 
               p = P_BLUR), # 模糊
        A.HorizontalFlip(p = P_HORIZONTALFLIP), # 水平翻轉
        A.VerticalFlip(p = P_VERTICALFLIP), # 垂直翻轉
        A.Flip(p = P_FLIP), # 水平和垂直翻轉
        A.Resize(height = RESIZE_HEIGHT[fold], 
                 width = RESIZE_WIDTH[fold], 
                 p = P_RESIZE), # 縮放
        A.RandomResizedCrop(height = RANDOMRESIZEDCROP_HEIGHT[fold], 
                            width = RANDOMRESIZEDCROP_WIDTH[fold], 
                            scale = RANDOMRESIZEDCROP_SCALE, 
                            p = P_RANDOMRESIZEDCROP), #隨機縮放剪裁
        A.RandomRotate90(p = P_RANDOMROTATE90), # 隨機旋轉90度
        A.ShiftScaleRotate(shift_limit = SHIFTSCALEROTATE_SHIFT_LIMIT, 
                           scale_limit = SHIFTSCALEROTATE_SCALE_LIMIT, 
                           rotate_limit = SHIFTSCALEROTATE_ROTATE_LIMIT, 
                           p = P_SHIFTSCALEROTATE), # 平移縮放旋轉
        A.ElasticTransform(alpha = ELATICTRANSFORM_ALPHA, 
                           sigma = ELATICTRANSFORM_SIGMA, 
                           alpha_affine = ELATICTRANSFORM_ALPHA_AFFINE, 
                           p = P_ELATICTRANSFORM), # 彈性變換
        A.GridDistortion(num_steps = GRIDDISTORTION_NUM_STEPS, 
                         p = P_GRIDDISTORTION), # 網格失真
        A.RandomBrightnessContrast(brightness_limit = RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT, 
                                   contrast_limit = RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT, 
                                   p = P_RANDOMBRIGHTNESSCONTRAST_CONTRAST), # 隨機亮度對比度
        A.HueSaturationValue(hue_shift_limit = HUESATURATIONVALUE_HUE_SHIFT_LIMIT, 
                             sat_shift_limit = HUESATURATIONVALUE_SAT_SHIFT_LIMIT, 
                             val_shift_limit = HUESATURATIONVALUE_VAL_SHIFT_LIMIT, 
                             p = P_HUESATURATIONVALUE), # 隨機色調飽和度值
        A.CLAHE(clip_limit = CLAHE_CLIP_LIMIT, 
                p = P_CLAHE), # 將對比度受限的自適應直方圖均衡化應用於輸入圖像
        A.Cutout(num_holes = COARSEDROPOUT_NUM_HOLES, 
                        max_h_size = COARSEDROPOUT_MAX_H_SIZE, 
                        max_w_size = COARSEDROPOUT_MAX_W_SIZE, 
                        p = P_COARSEDROPOUT), # 隨機在圖像上生成黑色矩形
        A.Normalize(
             mean = NORMALIZE_MEAN, 
             std = NORMALIZE_STD, 
            max_pixel_value = NORMALIZE_MAX_PIXEL_VALUE, 
            p = P_NORMALIZE), # 正規化。
        ToTensorV2(p = P_TOTENSORV2) # 歸一化
    ], p = P_TRAIN_TRANSFORMS)

In [17]:
def prepare_dataloader(fold, count):
    test_dataset = MyDataset(test_csv, transforms = get_test_transforms(fold))
    test_loader = DataLoader(test_dataset, batch_size = BATCH_SIZE[count], pin_memory = PIN_MEMORY, 
                                               shuffle = False, num_workers = NUM_WORKERS)
    return test_loader

In [18]:
# def inference_one_epoch(model, data_loader, device):
#     model.eval()
#     image_preds_all = []
#     # pbar = tqdm(enumerate(data_loader), total=len(data_loader))
#     with torch.no_grad():
#         for step, (imgs) in enumerate(data_loader):
#             imgs = imgs.to(device).float()

#             image_preds = model(imgs)   #output = model(input)
#             image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]
        
    
#     image_preds_all = np.concatenate(image_preds_all, axis=0)
#     return image_preds_all

In [19]:
def inference_process(count, fold, kf):
    if kf:
        print('Fold %i - image size %i with %s and batch size %i'%(fold+1,IMAGE_SIZE[count],LOAD_MODEL_NAME[count].upper(),BATCH_SIZE[count]))
    else:
        print('Image size %i with %s and batch_size %i'%(IMAGE_SIZE[count],LOAD_MODEL_NAME[count].upper(),BATCH_SIZE[count]))
    

#     build_model(count, custom_model_count, timm_model_count, base_model_count)
#     model = model.to(DEVICE)
#     model.eval()
    
    
#         if FOLD[0] >1:
#             count = 0
#             for x in LOAD_MODEL_PATH:
#                 LOAD_MODEL_PATH[count] = x + LOAD_MODEL_NAME[count]
#                 print(LOAD_MODEL_PATH[count])
#         else:
#             count = 0
#             for x in LOAD_MODEL_PATH:
#                 LOAD_MODEL_PATH[count] = x + LOAD_MODEL_NAME[count]
#                 print(LOAD_MODEL_PATH[count])
    
    
#     test_loader = prepare_dataloader(fold, count)

#     del test_loader
    gc.collect()

In [20]:
def main():
    try:
        print('Inference start')
        since = time.time()    
        for count in range(len(LOAD_MODEL_NAME)):
            if FOLD[count] > 1:
                for fold in enumerate(KF.split(np.arange(train_csv.shape[0]))):
                    inference_process(count, fold = fold, kf = True)
            else:
                inference_process(count, fold = 0, kf = False)
        time_elapsed = time.time() - since
        print('Inference complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    except Exception as exception:
        print(exception)
        raise

In [21]:
if __name__ == '__main__':
    main()

Inference start
Image size 224 with EFFICIENTNET_V2_B0 and batch_size 256
Image size 224 with EFFICIENTNET_V2_B0 and batch_size 256
Image size 224 with EFFICIENTNET_V2_B0 and batch_size 256
Inference complete in 0m 1s


# 7. 待辦事項<a class="anchor" id="7"></a>
[Back to Table of Contents](#0)
1. FOLD
1. TTA
1. ENSEMBLE